# TD3 2/2 : introduction d'un espace de représentation des mots (embedding)

Ce notebook a été développé dans le cours donné par J. Velcin et J. Cugliari sur le Deep Learning à l'Université de Lyon 2.

On commence par charger en mémoire les données spam diffusée à l'occasion du tutoriel de A. Gramfort et A. Mueller à SciPy 2017
https://github.com/amueller/scipy-2017-sklearn

In [1]:
import os

with open(os.path.join("SMSSpamCollection.txt")) as f:
    lines = [line.strip().split("\t") for line in f.readlines()]

text = [x[1] for x in lines]
y = [int(x[0] == "spam") for x in lines]

On procède différemment cette fois en ayant recours à une fonction de hachage, sans chercher pour le moment à construire explicitement la matrice documents x termes.

In [2]:
from tensorflow.keras.preprocessing.text import one_hot

# note that the index construction is based on hashing so that a small size will inscrease the risk of collision
vocab_size = 10000
encoded_docs = [one_hot(d, vocab_size) for d in text]

Init Plugin
Init Graph Optimizer
Init Kernel


Voyons ce que cela donne pour les premiers textes.

In [3]:
print(text[0:4])
print(encoded_docs[0:4])

['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...', 'Ok lar... Joking wif u oni...', "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's", 'U dun say so early hor... U c already then say...']
[[7124, 1471, 8153, 2822, 8991, 7443, 88, 7225, 5414, 4764, 109, 4826, 9852, 8179, 2976, 672, 1963, 5963, 9379, 2381], [6757, 7225, 194, 2613, 3704, 8822], [2276, 1636, 7225, 3226, 3231, 3028, 8506, 6187, 8518, 38, 7468, 4039, 3538, 9923, 1328, 3718, 2315, 38, 6187, 7458, 6187, 8585, 1636, 435, 5187, 4626, 7934, 4813, 8557, 8597, 8187], [3704, 3038, 5147, 2569, 6173, 5467, 3704, 7842, 2435, 9033, 5147]]


Pour construire un MLP, on a besoin d'une entrée de taille fixe. Deux options :
- utiliser un Bag of Words (BoW) avec des caractéristiques TF ou TFxIDF... (cf. TD 2)
- garder l'ordre des mots et traiter directement la séquence, en intégrant ou non une couche de type "word embedding"

In [4]:
#pad documents to a max length of 10 words

from tensorflow.keras.preprocessing.sequence import pad_sequences

max_length = 10
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs[0:4])

[[ 109 4826 9852 8179 2976  672 1963 5963 9379 2381]
 [6757 7225  194 2613 3704 8822    0    0    0    0]
 [8585 1636  435 5187 4626 7934 4813 8557 8597 8187]
 [3038 5147 2569 6173 5467 3704 7842 2435 9033 5147]]


On crée les jeux d'entraînement / test.

In [5]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y = train_test_split(padded_docs, y, 
                                                    train_size=0.7,
                                                    test_size=0.3,
                                                    random_state=123)

Puis on construit une architecture légèrement différente d'ANN.

On choisit ici de faire correspondre chaque mot (token), codé par son numéro (index), avec une ligne de la table enregistrant le codage dans un espace à k dimensions (cf. cours de text mining).

Ensuite les "embedding" sont concaténés (opération "flatten") avant d'être passés à la couche suivante.

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.layers import Embedding

def embedded_model(size_embedding):
    model = Sequential()
    # embedding layer
    model.add(Embedding(vocab_size, size_embedding, input_length=max_length))
    # let's concatenate the embedding vectors
    model.add(Flatten())
    model.add(Dense(8, activation='relu'))
    # simple binary classification
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
    return model

#padded_model = embedded_model(vocab_size)
padded_model = embedded_model(10)

# summarize the model
print(padded_model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 10, 10)            100000    
_________________________________________________________________
flatten_1 (Flatten)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 808       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 9         
Total params: 100,817
Trainable params: 100,817
Non-trainable params: 0
_________________________________________________________________
None


Le suite est identique aux précédents notebooks.

In [9]:
import numpy as np
train_y = np.array(train_y)
test_y = np.array(test_y)

In [10]:
# fit the model
padded_model.fit(train_X, train_y, epochs=10, verbose=1)

2021-12-07 10:43:36.785735: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-12-07 10:43:36.785927: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2021-12-07 10:43:36.922031: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Epoch 1/10
122/122 [==============================] - 1s 5ms/step - loss: 0.5273 - acc: 0.8582
Epoch 2/10
122/122 [==============================] - 1s 5ms/step - loss: 0.1881 - acc: 0.9136
Epoch 3/10
122/122 [==============================] - 1s 5ms/step - loss: 0.0924 - acc: 0.9772
Epoch 4/10
122/122 [==============================] - 1s 5ms/step - loss: 0.0451 - acc: 0.9897
Epoch 5/10
122/122 [==============================] - 1s 5ms/step - loss: 0.0260 - acc: 0.9936
Epoch 6/10
122/122 [==============================] - 1s 5ms/step - loss: 0.0158 - acc: 0.9969
Epoch 7/10
122/122 [==============================] - 1s 5ms/step - loss: 0.0101 - acc: 0.9985
Epoch 8/10
122/122 [==============================] - 1s 5ms/step - loss: 0.0071 - acc: 0.9990
Epoch 9/10
122/122 [==============================] - 1s 5ms/step - loss: 0.0048 - acc: 0.9995
Epoch 10/10
122/122 [==============================] - 1s 5ms/step - loss: 0.0035 - acc: 0.9995


In [9]:
loss, accuracy = padded_model.evaluate(test_X, test_y, verbose=0)
print('Accuracy: %f' % (accuracy*100))

2021-12-02 15:01:17.862061: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Accuracy: 98.386133
